# Info
Name: Nik Faiz Afiq bin Nik Ab Rahman<br>
Student ID: 17217542<br>
Course: WQD7006 Machine Learnig, Universiti Malaya

# Objective
1. Predict the coronavirus trend in each of the country
2. Listing the country that have worried trend( upward trend) and also countries that are doing well

# Preprocessing to do list
1. Merge all the data from the 3 table
2. Unpivot the table so all the date and data is in 1 column
3. Change the date format
4. Fill the NaN value with 0 or remove it


# Exploratory Analysis
1. Add cummulative cases, cummulative recovered and cummulative death
2. Some analysis might need to be done to achieve objective 2 - list the country that have upward trend and have downward trend
    - Some idea: https://stackoverflow.com/questions/24134894/identify-graph-uptrend-or-downtrend
    - Other idea: find gradient of last 7 days and see if it is positive or negative


# Modeling
1. Use the model to predict the number of new cases (X- date, Y-number of new cases)

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os


# Initial data import

In [53]:
root_folder = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

confirmed_df = pd.read_csv(root_folder+"time_series_covid19_confirmed_global.csv")
death_df = pd.read_csv(root_folder+"time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv(root_folder+"time_series_covid19_recovered_global.csv")



In [54]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48952,49161,49378,49621,49681,49817,50013,50190,50433,50655
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,50000,50637,51424,52004,52542,53003,53425,53814,54317,54827
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,93065,93507,93933,94371,94781,95203,95659,96069,96549,97007
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,7382,7446,7466,7519,7560,7577,7602,7633,7669,7699
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,16362,16407,16484,16562,16626,16644,16686,16802,16931,17029


In [55]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1995,2011,2025,2030,2047,2067,2082,2096,2117,2126
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1028,1040,1055,1066,1074,1088,1098,1111,1117,1125
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2623,2631,2640,2647,2659,2666,2675,2687,2696,2705
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,79,79,79,80,80,81,81,82,82,83
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,372,379,382,384,386,387,390,393,393,393


In [56]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38336,38475,38505,38540,38613,39006,39508,39585,39692,40359
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,25876,26381,26898,27426,27831,28121,28752,29249,29799,30276
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,61307,61700,62089,62487,62869,63260,63644,64020,64401,64777
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,6706,6819,6875,6919,6963,6997,7028,7073,7106,7171
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,8990,9194,9266,9345,9518,9592,9695,9729,9729,9729


# Preprocessing

1. Unpivot and merge all 3 data

In [57]:
confirmed_melt = pd.melt(confirmed_df, id_vars = ['Province/State', 'Country/Region','Lat','Long'],
                         value_vars = confirmed_df.columns[4:],
                         var_name = "Date",value_name = 'Confirmed')
death_melt = pd.melt(death_df, id_vars = ['Province/State', 'Country/Region','Lat','Long'],
                         value_vars = death_df.columns[4:],
                         var_name = "Date",value_name = 'Death')
recovered_melt = pd.melt(recovered_df, id_vars = ['Province/State', 'Country/Region','Lat','Long'],
                         value_vars = recovered_df.columns[4:],
                         var_name = "Date",value_name = 'Recovered')

confirmed_melt.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


2. Merging

In [58]:
merged1 = pd.merge(left = confirmed_melt ,right  = death_melt,how= "outer",
                  on = ['Province/State', 'Country/Region','Date','Lat','Long'])
merged_df = pd.merge(left = merged1 ,right  = recovered_melt,how= "outer",
                  on = ['Province/State', 'Country/Region','Date','Lat','Long'])
print(merged_df.shape)
merged_df.head()

(93626, 8)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Death,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0.0,0.0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0.0,0.0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0.0,0.0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0.0,0.0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0.0,0.0,0.0


3. handling missing data

In [59]:
# percentage of missing data
merged_df.isnull().sum()/merged_df.shape[0]

Province/State    0.693141
Country/Region    0.000000
Lat               0.003610
Long              0.003610
Date              0.000000
Confirmed         0.021661
Death             0.021661
Recovered         0.075812
dtype: float64

In [60]:
# missing value of recovered with 0
merged_df['Recovered'] = merged_df['Recovered'].fillna(0)  

# change the values to integer
merged_df['Recovered'] = merged_df['Recovered'].astype('int') 

# missing value of province/state with ' '
merged_df['Province/State'] = merged_df['Recovered'].fillna(' ')

# Add active column
merged_df['Active'] = merged_df['Confirmed']-merged_df['Death'] - merged_df['Recovered'] 

In [61]:
merged_df.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Death,Recovered,Active
93621,367,China,37.8957,114.9042,12/24/20,NaN,NaN,367,NaN
93622,1270,China,33.8820,113.6140,12/24/20,NaN,NaN,1270,NaN
93623,16015,Mozambique,-18.6657,35.5296,12/24/20,NaN,NaN,16015,NaN
93624,4997,Syria,34.8021,38.9968,12/24/20,NaN,NaN,4997,NaN
93625,31,Timor-Leste,-8.8742,125.7275,12/24/20,NaN,NaN,31,NaN


3. Convert date types

In [62]:
print(merged_df['Date'].describe())

merged_df['Date'] = pd.to_datetime(merged_df['Date']).dt.normalize()

print(merged_df['Date'].describe(datetime_is_numeric=True))
merged_df['Date']

count      93626
unique       338
top       9/6/20
freq         277
Name: Date, dtype: object
count                            93626
mean     2020-07-08 12:00:00.000023040
min                2020-01-22 00:00:00
25%                2020-04-15 00:00:00
50%                2020-07-08 12:00:00
75%                2020-10-01 00:00:00
max                2020-12-24 00:00:00
Name: Date, dtype: object


0       2020-01-22
1       2020-01-22
2       2020-01-22
3       2020-01-22
4       2020-01-22
           ...    
93621   2020-12-24
93622   2020-12-24
93623   2020-12-24
93624   2020-12-24
93625   2020-12-24
Name: Date, Length: 93626, dtype: datetime64[ns]

4. Save file to csv

In [63]:
merged_df.to_csv('data/covid19_cleaned.csv',index=False)

In [64]:
print(merged_df.shape)
merged_df.head()

(93626, 9)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Death,Recovered,Active
0,0,Afghanistan,33.93911,67.709953,2020-01-22,0.0,0.0,0,0.0
1,0,Albania,41.15330,20.168300,2020-01-22,0.0,0.0,0,0.0
2,0,Algeria,28.03390,1.659600,2020-01-22,0.0,0.0,0,0.0
3,0,Andorra,42.50630,1.521800,2020-01-22,0.0,0.0,0,0.0
4,0,Angola,-11.20270,17.873900,2020-01-22,0.0,0.0,0,0.0


#### New Cases column

In [65]:
# grouped the data based on the country
new = merged_df.groupby(['Date','Country/Region'])['Confirmed','Death','Recovered','Active'].sum().reset_index()
print(new.shape)
new.head()

(64558, 6)


<ipython-input-65-cbcc3bf0d952>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  new = merged_df.groupby(['Date','Country/Region'])['Confirmed','Death','Recovered','Active'].sum().reset_index()


,Date,Country/Region,Confirmed,Death,Recovered,Active
0,2020-01-22,Afghanistan,0.0,0.0,0,0.0
1,2020-01-22,Albania,0.0,0.0,0,0.0
2,2020-01-22,Algeria,0.0,0.0,0,0.0
3,2020-01-22,Andorra,0.0,0.0,0,0.0
4,2020-01-22,Angola,0.0,0.0,0,0.0


Creating new columns dataframe and merged it

In [66]:
new_col = merged_df.groupby(['Country/Region','Date'])['Confirmed','Death','Recovered'].sum().diff().reset_index()

new_col.columns = ['Country/Region', 'Date', 'New Confirmed', 'New Death', 'New Recovered']

# fix first row of the countries
first_rows = (new_col['Country/Region'] != new_col["Country/Region"].shift(1))
new_col.loc[first_rows,'New Confirmed'] = 0
new_col.loc[first_rows,'New Death'] = 0
new_col.loc[first_rows,'New Recovered'] = 0

print(new.head())
print(new_col.head())


# Merge the new values
df = pd.merge(new,new_col,on=['Country/Region','Date'], how = 'left')
df.head()

        Date Country/Region  Confirmed  Death  Recovered  Active
0 2020-01-22    Afghanistan        0.0    0.0          0     0.0
1 2020-01-22        Albania        0.0    0.0          0     0.0
2 2020-01-22        Algeria        0.0    0.0          0     0.0
3 2020-01-22        Andorra        0.0    0.0          0     0.0
4 2020-01-22         Angola        0.0    0.0          0     0.0
  Country/Region       Date  New Confirmed  New Death  New Recovered
0    Afghanistan 2020-01-22            0.0        0.0            0.0
1    Afghanistan 2020-01-23            0.0        0.0            0.0
2    Afghanistan 2020-01-24            0.0        0.0            0.0
3    Afghanistan 2020-01-25            0.0        0.0            0.0
4    Afghanistan 2020-01-26            0.0        0.0            0.0


<ipython-input-66-2c6ecae3e656>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  new_col = merged_df.groupby(['Country/Region','Date'])['Confirmed','Death','Recovered'].sum().diff().reset_index()


,Date,Country/Region,Confirmed,Death,Recovered,Active,New Confirmed,New Death,New Recovered
0,2020-01-22,Afghanistan,0.0,0.0,0,0.0,0.0,0.0,0.0
1,2020-01-22,Albania,0.0,0.0,0,0.0,0.0,0.0,0.0
2,2020-01-22,Algeria,0.0,0.0,0,0.0,0.0,0.0,0.0
3,2020-01-22,Andorra,0.0,0.0,0,0.0,0.0,0.0,0.0
4,2020-01-22,Angola,0.0,0.0,0,0.0,0.0,0.0,0.0


In [67]:
# Fix datatype for new column
df[ ['New Confirmed', 'New Death', 'New Recovered']] = df[ ['New Confirmed', 'New Death', 'New Recovered']].astype('int')


In [68]:
df

,Date,Country/Region,Confirmed,Death,Recovered,Active,New Confirmed,New Death,New Recovered
0,2020-01-22,Afghanistan,0.0,0.0,0,0.0,0,0,0
1,2020-01-22,Albania,0.0,0.0,0,0.0,0,0,0
2,2020-01-22,Algeria,0.0,0.0,0,0.0,0,0,0
3,2020-01-22,Andorra,0.0,0.0,0,0.0,0,0,0
4,2020-01-22,Angola,0.0,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
64553,2020-12-24,Vietnam,1432.0,35.0,1281,116.0,11,0,0
64554,2020-12-24,West Bank and Gaza,129080.0,1243.0,105233,22604.0,1704,17,2291
64555,2020-12-24,Yemen,2092.0,606.0,1384,102.0,5,0,0
64556,2020-12-24,Zambia,19234.0,381.0,18036,817.0,112,1,67


Export data to csv

In [69]:
df.to_csv('data/covid19_df.csv',index=False)

In [70]:
# NA check
df.isnull().sum()

Date              0
Country/Region    0
Confirmed         0
Death             0
Recovered         0
Active            0
New Confirmed     0
New Death         0
New Recovered     0
dtype: int64

## Latest Data

In [71]:
# latest df
latest_df = df[df["Date"] == df["Date"].max()]
latest_df.reset_index(drop= True,inplace=True)
latest_df

,Date,Country/Region,Confirmed,Death,Recovered,Active,New Confirmed,New Death,New Recovered
0,2020-12-24,Afghanistan,50655.0,2126.0,40359,8170.0,222,9,667
1,2020-12-24,Albania,54827.0,1125.0,30276,23426.0,510,8,477
2,2020-12-24,Algeria,97007.0,2705.0,64777,29525.0,458,9,376
3,2020-12-24,Andorra,7699.0,83.0,7171,445.0,30,1,65
4,2020-12-24,Angola,17029.0,393.0,9729,6907.0,98,0,0
...,...,...,...,...,...,...,...,...,...
186,2020-12-24,Vietnam,1432.0,35.0,1281,116.0,11,0,0
187,2020-12-24,West Bank and Gaza,129080.0,1243.0,105233,22604.0,1704,17,2291
188,2020-12-24,Yemen,2092.0,606.0,1384,102.0,5,0,0
189,2020-12-24,Zambia,19234.0,381.0,18036,817.0,112,1,67


## World Data

In [72]:
world_df = df.groupby(['Date']).sum().astype('int').reset_index()
world_df

,Date,Confirmed,Death,Recovered,Active,New Confirmed,New Death,New Recovered
0,2020-01-22,555,17,28,510,0,0,0
1,2020-01-23,654,18,30,606,99,1,2
2,2020-01-24,941,26,36,879,287,8,6
3,2020-01-25,1434,42,39,1353,493,16,3
4,2020-01-26,2118,56,52,2010,684,14,13
...,...,...,...,...,...,...,...,...
333,2020-12-20,76823486,1693205,43367172,32207191,534444,7679,273430
334,2020-12-21,77364641,1702596,43707973,32405680,541155,9391,340801
335,2020-12-22,78011432,1717055,44069203,32685041,646791,14459,361230
336,2020-12-23,78704434,1730663,44393791,33046597,693002,13608,324588


## Statistics

In [73]:
# table for top 10 countries
col = 'Confirmed'
latest_df.sort_values(col,ascending = False).head(10)

,Date,Country/Region,Confirmed,Death,Recovered,Active,New Confirmed,New Death,New Recovered
177,2020-12-24,US,18650454.0,329023.0,0,18321431.0,192081,2899,0
79,2020-12-24,India,10146845.0,147092.0,9717834,281919.0,23067,336,24661
23,2020-12-24,Brazil,7423945.0,189982.0,6512717,721246.0,58428,762,0
141,2020-12-24,Russia,2934695.0,52434.0,2347700,534561.0,29499,624,26338
62,2020-12-24,France,2584333.0,62389.0,194559,2327385.0,21718,291,1367
181,2020-12-24,United Kingdom,2195144.0,69732.0,4821,2120591.0,39148,575,98
176,2020-12-24,Turkey,2100712.0,19115.0,1935292,146305.0,18102,254,33985
85,2020-12-24,Italy,2009317.0,70900.0,1344785,593632.0,18039,505,22718
161,2020-12-24,Spain,1854951.0,49824.0,150376,1654751.0,12662,126,0
66,2020-12-24,Germany,1630596.0,29330.0,1220277,380989.0,26467,421,22805
